In [27]:
# Core imports
import os
import glob
from dotenv import load_dotenv
import gradio as gr

# LangChain and ML imports
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader, UnstructuredHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Visualization imports (optional)
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go


In [28]:
# Model configuration - using cost-effective model
MODEL = "gpt-4o-mini"
db_name = "vector_db"

# Load environment variables
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

# Verify API key is loaded
if not os.getenv('OPENAI_API_KEY'):
    print("⚠️ Warning: OpenAI API key not found!")
else:
    print("✅ OpenAI API key loaded successfully")


✅ OpenAI API key loaded successfully


In [29]:
# Document loading functions for different platforms
def load_txt_folder(path, site_tag):
    """Load text files from a directory"""
    try:
        loader = DirectoryLoader(
            path=path,
            glob="**/*.txt",
            loader_cls=lambda p: TextLoader(p, encoding="utf-8")
        )
        docs = loader.load()
        for d in docs:
            d.metadata["source_site"] = site_tag
        print(f"✅ Loaded {len(docs)} TXT files from {site_tag}")
        return docs
    except Exception as e:
        print(f"❌ Error loading TXT from {path}: {str(e)}")
        return []

def load_html_folder(path, site_tag):
    """Load HTML files from a directory"""
    try:
        loader = DirectoryLoader(
            path=path,
            glob="**/*.html",
            loader_cls=UnstructuredHTMLLoader
        )
        docs = loader.load()
        for d in docs:
            d.metadata["source_site"] = site_tag
        print(f"✅ Loaded {len(docs)} HTML files from {site_tag}")
        return docs
    except Exception as e:
        print(f"❌ Error loading HTML from {path}: {str(e)}")
        return []

def load_pdf_folder(path, site_tag):
    """Load PDF files from a directory"""
    try:
        loader = DirectoryLoader(
            path=path,
            glob="**/*.pdf",
            loader_cls=PyPDFLoader
        )
        docs = loader.load()
        for d in docs:
            d.metadata["source_site"] = site_tag
        print(f"✅ Loaded {len(docs)} PDF files from {site_tag}")
        return docs
    except Exception as e:
        print(f"❌ Error loading PDF from {path}: {str(e)}")
        return []


In [30]:
# Root folders – create directories if they don't exist
ROOT_MOSDAC_PDF = "data_ingestion/mosdac_data/documents"
ROOT_MOSDAC_WEB = "data_ingestion/mosdac_data/web_pages"
ROOT_VEDAS_WEB = "data_ingestion/vedas_data/web_pages"
ROOT_BHUVAN_WEB = "data_ingestion/bhuvan_data/web_pages"

# Create directories if they don't exist
directories = [ROOT_MOSDAC_PDF, ROOT_MOSDAC_WEB, ROOT_VEDAS_WEB, ROOT_BHUVAN_WEB]
for directory in directories:
    os.makedirs(directory, exist_ok=True)
    print(f"📁 Directory ready: {directory}")

# Load documents from all platforms
print("\n🔄 Loading documents from all platforms...")

mosdac_pdfs = load_pdf_folder(ROOT_MOSDAC_PDF, "mosdac")
mosdac_txt = load_txt_folder(ROOT_MOSDAC_WEB, "mosdac")
mosdac_html = load_html_folder(ROOT_MOSDAC_WEB, "mosdac")

vedas_txt = load_txt_folder(ROOT_VEDAS_WEB, "vedas")
vedas_html = load_html_folder(ROOT_VEDAS_WEB, "vedas")

bhuvan_txt = load_txt_folder(ROOT_BHUVAN_WEB, "bhuvan")
bhuvan_html = load_html_folder(ROOT_BHUVAN_WEB, "bhuvan")

# Combine all documents
documents_all = mosdac_pdfs + mosdac_txt + mosdac_html + vedas_txt + vedas_html + bhuvan_txt + bhuvan_html

print(f"\n📊 Total documents loaded: {len(documents_all)}")
if documents_all:
    print("📋 Sample metadata:", documents_all[0].metadata)
    
    # Count by platform
    platform_counts = {}
    for doc in documents_all:
        platform = doc.metadata.get('source_site', 'unknown')
        platform_counts[platform] = platform_counts.get(platform, 0) + 1
    
    print("📈 Documents by platform:")
    for platform, count in platform_counts.items():
        print(f"   {platform.upper()}: {count} documents")
else:
    print("⚠️ No documents found! Please add files to the data directories.")


📁 Directory ready: data_ingestion/mosdac_data/documents
📁 Directory ready: data_ingestion/mosdac_data/web_pages
📁 Directory ready: data_ingestion/vedas_data/web_pages
📁 Directory ready: data_ingestion/bhuvan_data/web_pages

🔄 Loading documents from all platforms...


Multiple definitions in dictionary at byte 0x30f8cb for key /Im1027
Multiple definitions in dictionary at byte 0x30f8dc for key /Im1027


✅ Loaded 1265 PDF files from mosdac
✅ Loaded 220 TXT files from mosdac
✅ Loaded 0 HTML files from mosdac
✅ Loaded 233 TXT files from vedas
✅ Loaded 0 HTML files from vedas
✅ Loaded 6 TXT files from bhuvan
✅ Loaded 0 HTML files from bhuvan

📊 Total documents loaded: 1724
📋 Sample metadata: {'producer': 'Nitro PDF PrimoPDF', 'creator': 'PrimoPDF http://www.primopdf.com', 'creationdate': '2012-03-20T14:49:18-05:30', 'moddate': '2012-03-20T14:49:18-05:30', 'title': 'Microsoft Word - Analysed-Winds', 'author': 'admin', 'source': 'data_ingestion\\mosdac_data\\documents\\Analysed-Winds.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1', 'source_site': 'mosdac'}
📈 Documents by platform:
   MOSDAC: 1485 documents
   VEDAS: 233 documents
   BHUVAN: 6 documents


In [31]:
# Text splitting with optimized parameters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""],
)

print("🔄 Splitting documents into chunks...")
chunks_all = text_splitter.split_documents(documents_all)
print(f"📊 Total chunks created: {len(chunks_all)}")

# Analyze chunks by platform
if chunks_all:
    chunk_counts = {}
    for chunk in chunks_all:
        platform = chunk.metadata.get('source_site', 'unknown')
        chunk_counts[platform] = chunk_counts.get(platform, 0) + 1
    
    print("📈 Chunks by platform:")
    for platform, count in chunk_counts.items():
        print(f"   {platform.upper()}: {count} chunks")
    
    # Show sample chunk
    first_chunk = chunks_all[0]
    print(f"\n📋 Sample chunk metadata: {first_chunk.metadata}")
    print(f"📝 Sample content preview: {first_chunk.page_content[:200]}...")


🔄 Splitting documents into chunks...
📊 Total chunks created: 4596
📈 Chunks by platform:
   MOSDAC: 3956 chunks
   VEDAS: 634 chunks
   BHUVAN: 6 chunks

📋 Sample chunk metadata: {'producer': 'Nitro PDF PrimoPDF', 'creator': 'PrimoPDF http://www.primopdf.com', 'creationdate': '2012-03-20T14:49:18-05:30', 'moddate': '2012-03-20T14:49:18-05:30', 'title': 'Microsoft Word - Analysed-Winds', 'author': 'admin', 'source': 'data_ingestion\\mosdac_data\\documents\\Analysed-Winds.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1', 'source_site': 'mosdac'}
📝 Sample content preview: GLOBAL ANALYSED OCEAN SURFACE WIND PRODUCTS 
 
 
Description 
The analysed winds have been generated at 0.5 
0×0.5 0 interval over the global oceans. For the generation of 
these analysed winds produc...


In [32]:
# Initialize embeddings and vector store
print("🔄 Setting up vector store...")

embeddings = OpenAIEmbeddings()

# Optional: start from a clean collection if you want to rebuild
if os.path.exists(db_name):
    print("🗑️ Removing existing vector store...")
    try:
        old_store = Chroma(persist_directory=db_name, embedding_function=embeddings)
        old_store.delete_collection()
        print("✅ Old vector store removed")
    except Exception as e:
        print(f"⚠️ Could not remove old store: {e}")

print("🔄 Creating new vector store...")
vectorstore = Chroma(persist_directory=db_name, embedding_function=embeddings)

# Batch insert with progress tracking
if chunks_all:
    BATCH_SIZE = 64
    total_chunks = len(chunks_all)
    
    print(f"📊 Inserting {total_chunks} chunks in batches of {BATCH_SIZE}...")
    
    for i in range(0, total_chunks, BATCH_SIZE):
        batch = chunks_all[i:i+BATCH_SIZE]
        vectorstore.add_documents(batch)
        
        progress = min(i + BATCH_SIZE, total_chunks)
        percentage = (progress / total_chunks) * 100
        print(f"✅ Progress: {progress}/{total_chunks} chunks ({percentage:.1f}%)")
    
    final_count = vectorstore._collection.count()
    print(f"🎉 Vector store created successfully!")
    print(f"📊 Final collection size: {final_count}")
else:
    print("❌ No chunks to insert into vector store!")


🔄 Setting up vector store...
🗑️ Removing existing vector store...
✅ Old vector store removed
🔄 Creating new vector store...
📊 Inserting 4596 chunks in batches of 64...
✅ Progress: 64/4596 chunks (1.4%)
✅ Progress: 128/4596 chunks (2.8%)
✅ Progress: 192/4596 chunks (4.2%)
✅ Progress: 256/4596 chunks (5.6%)
✅ Progress: 320/4596 chunks (7.0%)
✅ Progress: 384/4596 chunks (8.4%)
✅ Progress: 448/4596 chunks (9.7%)
✅ Progress: 512/4596 chunks (11.1%)
✅ Progress: 576/4596 chunks (12.5%)
✅ Progress: 640/4596 chunks (13.9%)
✅ Progress: 704/4596 chunks (15.3%)
✅ Progress: 768/4596 chunks (16.7%)
✅ Progress: 832/4596 chunks (18.1%)
✅ Progress: 896/4596 chunks (19.5%)
✅ Progress: 960/4596 chunks (20.9%)
✅ Progress: 1024/4596 chunks (22.3%)
✅ Progress: 1088/4596 chunks (23.7%)
✅ Progress: 1152/4596 chunks (25.1%)
✅ Progress: 1216/4596 chunks (26.5%)
✅ Progress: 1280/4596 chunks (27.9%)
✅ Progress: 1344/4596 chunks (29.2%)
✅ Progress: 1408/4596 chunks (30.6%)
✅ Progress: 1472/4596 chunks (32.0%)
✅ Pr

In [33]:
# Initialize LLM and retriever
print("🔄 Setting up RAG chain...")

llm = ChatOpenAI(temperature=0.7, model_name=MODEL)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# Enhanced prompt template
rag_prompt_template = """You are ASTROGEO AI, a specialized assistant for Indian space and earth observation data platforms.

You have access to information from three major platforms:
- MOSDAC: Meteorological & Oceanographic Satellite Data Archival Centre
- VEDAS: Visualization of Earth observation Data and Archival System  
- BHUVAN: Indian Geo-platform for visualization and analysis

Instructions:
1. Answer questions based on the provided context documents
2. If information is available in the documents, provide detailed, comprehensive responses
3. Always mention which platform(s) the information comes from (MOSDAC, VEDAS, or BHUVAN)
4. If documents don't contain the answer, use general knowledge but clearly state this
5. Be helpful, accurate, and technical when appropriate
6. For data access questions, provide specific steps and requirements

Context from documents:
{context}

Chat History:
{chat_history}

User Question: {question}

Response:"""

RAG_PROMPT = PromptTemplate.from_template(rag_prompt_template)

# Create the conversational RAG chain
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": RAG_PROMPT},
    return_source_documents=True
)

print("✅ RAG chain setup complete!")


🔄 Setting up RAG chain...
✅ RAG chain setup complete!


In [34]:
def is_relevant(question, docs):
    """Check if retrieved documents are relevant to the question"""
    if not docs:
        return False
    
    # Take the first document for relevance check
    doc_content = docs[0].page_content[:1500] if docs else ""
    
    relevance_prompt = f"""Analyze if the following document content is relevant to answer the user's question.
    Answer only "Yes" or "No".
    
    Question: {question}
    
    Document Content: {doc_content}
    
    Relevance Assessment:"""
    
    try:
        check = llm.invoke(relevance_prompt)
        return "Yes" in check.content
    except Exception as e:
        print(f"⚠️ Relevance check error: {e}")
        return True  # Default to using RAG if check fails

# Fallback prompt for general knowledge responses
fallback_prompt_template = """You are ASTROGEO AI, a helpful assistant specializing in Indian space and earth observation platforms (MOSDAC, VEDAS, BHUVAN).

The user's question doesn't seem to be directly related to the documents in your knowledge base, so please answer using your general knowledge while staying within your area of expertise.

If the question is about:
- Satellite data, remote sensing, or earth observation
- Indian space programs (ISRO)
- Weather and climate data
- GIS and mapping
- Data access and processing

Please provide a helpful response. If it's completely outside your domain, politely redirect to your specializations.

Question: {question}

Response:"""

FALLBACK_PROMPT = PromptTemplate.from_template(fallback_prompt_template)

def smart_bot(message, history):
    """Main chatbot function with smart routing"""
    try:
        # Retrieve relevant documents
        docs = retriever.get_relevant_documents(message)
        
        # Check relevance and route accordingly
        if docs and is_relevant(message, docs):
            # Use RAG chain for document-based response
            result = rag_chain.invoke({"question": message})
            
            # Add source information
            sources = set()
            if hasattr(result, 'source_documents') and result['source_documents']:
                for doc in result['source_documents']:
                    source_site = doc.metadata.get('source_site', 'unknown')
                    sources.add(source_site.upper())
            
            response = result["answer"]
            if sources:
                response += f"\n\n*Sources: {', '.join(sources)}*"
            
            return response
        else:
            # Use fallback for general knowledge
            chain = FALLBACK_PROMPT | llm
            result = chain.invoke({"question": message})
            return result.content + "\n\n*Note: This response is based on general knowledge as no relevant documents were found in the knowledge base.*"
            
    except Exception as e:
        return f"❌ Sorry, I encountered an error: {str(e)}. Please try rephrasing your question."

print("✅ Smart bot function ready!")


✅ Smart bot function ready!


In [35]:
# Create and launch the Gradio interface
print("🚀 Launching ASTROGEO AI Chat Interface...")

# Custom CSS for better appearance
custom_css = """
.gradio-container {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif !important;
}
"""

# Create the chat interface
interface = gr.ChatInterface(
    fn=smart_bot,
    title="🛰️ ASTROGEO AI — Multi-Platform Earth Observation Assistant",
    description="""
    **Powered by MOSDAC + VEDAS + BHUVAN Knowledge Base**
    
    Ask me about:
    • MOSDAC satellite data and services
    • VEDAS earth observation and visualization
    • BHUVAN geospatial data and mapping
    • Data access procedures and formats
    • Indian space and remote sensing programs
    """,
    theme="soft",
    css=custom_css,
    examples=[
        "What types of data are available in MOSDAC?",
        "How do I access satellite imagery from VEDAS?",
        "What mapping services does BHUVAN provide?",
        "How can I download INSAT-3D data?",
        "What are the data formats supported by these platforms?",
        "Tell me about Indian earth observation satellites"
    ],
    cache_examples=False,
    analytics_enabled=False
)

# Launch the interface
try:
    interface.launch(
        share=True,  # Creates public link
        server_name="0.0.0.0",  # Makes it accessible on network
        server_port=7860,  # Default Gradio port
        show_error=True
    )
    print("✅ Interface launched successfully!")
except Exception as e:
    print(f"❌ Launch error: {e}")
    # Fallback launch without share
    interface.launch(show_error=True)


🚀 Launching ASTROGEO AI Chat Interface...


c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


❌ Launch error: Cannot find empty port in range: 7860-7860. You can specify a different port by setting the GRADIO_SERVER_PORT environment variable or passing the `server_port` parameter to `launch()`.
* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [48]:
# Test the system with sample queries
def test_system():
    """Test the chatbot with sample queries"""
    test_queries = [
        "What is MOSDAC?",
        "How do I access VEDAS data?",
        "What services does BHUVAN provide?",
        "Tell me about INSAT satellites",
        "What is the weather like today?"  # This should trigger fallback
    ]
    
    print("🧪 Testing the system with sample queries...")
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n{'='*50}")
        print(f"Test {i}: {query}")
        print('='*50)
        
        try:
            response = smart_bot(query, [])
            print(f"Response: {response[:300]}...")
        except Exception as e:
            print(f"❌ Error: {e}")

# Uncomment the line below to run tests
# test_system()

print("\n🎉 ASTROGEO AI is ready!")
print("📊 System Statistics:")
print(f"   • Total documents: {len(documents_all)}")
print(f"   • Total chunks: {len(chunks_all)}")
print(f"   • Vector store size: {vectorstore._collection.count()}")
print(f"   • Model: {MODEL}")



🎉 ASTROGEO AI is ready!
📊 System Statistics:
   • Total documents: 1724
   • Total chunks: 4596
   • Vector store size: 4596
   • Model: gpt-4o-mini


In [67]:
# Install additional packages for image generation
!pip install huggingface_hub --upgrade --quiet
!pip install Pillow --upgrade --quiet

print("✅ Additional dependencies installed!")
# Add HF_TOKEN for image generation
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-hugging-face-token-here')

# Verify HF token
if not os.getenv('HF_TOKEN'):
    print("⚠️ Warning: HF_TOKEN not found!")
else:
    print("✅ HF_TOKEN loaded successfully")



✅ Additional dependencies installed!
✅ HF_TOKEN loaded successfully


In [ ]:
from huggingface_hub import InferenceClient
from IPython.display import Image, display
import base64
from io import BytesIO
from PIL import Image as PILImage


class SimpleImageGenerator:
    def __init__(self, hf_token):
        self.client = InferenceClient(provider="fal-ai", api_key=hf_token)
        
    def generate_image(self, prompt):
        try:
            # Generate image with FLUX model
            response = self.client.text_to_image(
                prompt=f"High-resolution astronomical visualization of {prompt}, cosmic photography, space telescope quality, detailed celestial objects",
                model="black-forest-labs/FLUX.1-dev"
            )
            return response, "✅ Generated successfully"
        except Exception as e:
            return None, f"❌ Generation failed: {str(e)}"

# Initialize image generator
if os.getenv('HF_TOKEN'):
    image_gen = SimpleImageGenerator(os.getenv('HF_TOKEN'))
    print("✅ Image generator ready!")
else:
    image_gen = None
    print("❌ Image generator not available - missing HF_TOKEN")


class ASTROGEOImageGenerator:
    """Enhanced image generator for ASTROGEO AI with dual specialty support"""
    def create_image_enabled_interface():
    """Create interface with working image generation"""
    
    custom_css = """
    .gradio-container { max-width: 1400px !important; margin: 0 auto !important; }
    .astronomy-mode { background: linear-gradient(45deg, #1a1a3e, #4a90e2); color: white; padding: 15px; border-radius: 10px; text-align: center; }
    """
    
    with gr.Blocks(css=custom_css, title="🛰️ ASTROGEO AI + Image Gen") as interface:
        
        gr.HTML("""
        <div style='text-align: center; padding: 25px; background: linear-gradient(45deg, #1e3c72, #27ae60); border-radius: 15px; margin-bottom: 20px;'>
            <h1 style='color: white; font-size: 2.5em;'>🛰️ ASTROGEO AI + IMAGE GENERATION</h1>
            <p style='color: white; font-size: 1.2em;'>Text + Image Generation Enabled</p>
        </div>
        """)
        
        with gr.Row():
            with gr.Column(scale=2):
                chatbot = gr.Chatbot(label="🛰️ ASTROGEO AI", height=500)
                
                with gr.Row():
                    msg = gr.Textbox(
                        placeholder="Ask questions or request images (e.g., 'generate image of black hole')",
                        lines=2, scale=4
                    )
                    send_btn = gr.Button("🚀 Send", variant="primary")
            
            with gr.Column(scale=1):
                image_output = gr.Image(label="Generated Image", visible=False, height=400)
                status_output = gr.HTML("Ready for questions...")
        
        def respond_with_images(message, history):
            if not message.strip():
                return history, "", None, gr.update(visible=False), "Please enter a message"
            
            try:
                # Get text response from your existing smart_bot
                text_response = smart_bot(message, history)
                
                # Check if image generation is requested
                image_triggers = ["generate", "image", "picture", "show me", "visualize", "create"]
                needs_image = any(trigger in message.lower() for trigger in image_triggers)
                
                if needs_image and image_gen:
                    # Generate image
                    generated_image, image_status = image_gen.generate_image(message)
                    
                    if generated_image:
                        enhanced_response = f"{text_response}\n\n🖼️ **{image_status}**"
                        history.append((message, enhanced_response))
                        
                        return (
                            history, 
                            "", 
                            generated_image, 
                            gr.update(visible=True),
                            "<div class='astronomy-mode'><h3>🌌 IMAGE GENERATED!</h3></div>"
                        )
                    else:
                        enhanced_response = f"{text_response}\n\n❌ **{image_status}**"
                        history.append((message, enhanced_response))
                        return history, "", None, gr.update(visible=False), image_status
                else:
                    # Text-only response
                    history.append((message, text_response))
                    return history, "", None, gr.update(visible=False), "Text response provided"
                    
            except Exception as e:
                error_msg = f"❌ Error: {str(e)}"
                history.append((message, error_msg))
                return history, "", None, gr.update(visible=False), f"Error: {str(e)}"
        
        # Connect events
        send_btn.click(
            respond_with_images,
            [msg, chatbot],
            [chatbot, msg, image_output, image_output, status_output]
        )
        
        msg.submit(
            respond_with_images,
            [msg, chatbot],
            [chatbot, msg, image_output, image_output, status_output]
        )
    
    return interface

# Create the enhanced interface
working_interface = create_image_enabled_interface()
print("✅ Image-enabled interface created!")

    
    def __init__(self, hf_token):
        self.client = InferenceClient(
            provider="fal-ai",
            api_key=hf_token
        )
        
        # Available models for different purposes
        self.models = {
            "flux": "black-forest-labs/FLUX.1-dev",  # Best quality
            "stable_diffusion": "stabilityai/stable-diffusion-xl-base-1.0",  # Reliable
            "recraft": "fal-ai/recraft-v3"  # Technical diagrams
        }
        
        # Keywords for smart classification
        self.astronomy_keywords = [
            "star", "galaxy", "nebula", "planet", "cosmos", "universe", "telescope", 
            "constellation", "black hole", "solar system", "asteroid", "comet",
            "space", "celestial", "astronomical", "cosmic", "lunar", "mars", "jupiter"
        ]
        
        self.geospatial_keywords = [
            "satellite", "earth", "landsat", "sentinel", "modis", "radar", "lidar",
            "remote sensing", "gis", "mapping", "terrain", "land use", "forest",
            "agriculture", "urban", "mosdac", "vedas", "bhuvan", "insat", "weather"
        ]
    
    def classify_image_type(self, prompt):
        """Classify if prompt is astronomy or geospatial focused"""
        prompt_lower = prompt.lower()
        
        astro_score = sum(1 for keyword in self.astronomy_keywords if keyword in prompt_lower)
        geo_score = sum(1 for keyword in self.geospatial_keywords if keyword in prompt_lower)
        
        if astro_score > geo_score:
            return "astronomy"
        elif geo_score > astro_score:
            return "geospatial"
        else:
            return "geospatial"  # Default to geospatial for earth observation focus
    
    def enhance_prompt(self, prompt, specialty="auto"):
        """Enhance prompt based on specialty"""
        if specialty == "auto":
            specialty = self.classify_image_type(prompt)
        
        if specialty == "astronomy":
            enhanced = f"High-resolution astronomical image of {prompt}, deep space photography, cosmic details, professional space telescope quality, detailed celestial objects, 4K resolution"
        else:  # geospatial
            enhanced = f"High-resolution satellite imagery of {prompt}, earth observation style, remote sensing visualization, technical accuracy, geographic detail, professional satellite photography"
        
        return enhanced, specialty
    
    def generate_image(self, prompt, model="flux", enhance=True):
        """Generate image with enhanced prompts and error handling"""
        try:
            # Enhance prompt if requested
            if enhance:
                enhanced_prompt, detected_specialty = self.enhance_prompt(prompt)
                specialty_info = f"🌌 ASTRONOMY" if detected_specialty == "astronomy" else "🛰️ GEOSPATIAL"
            else:
                enhanced_prompt = prompt
                specialty_info = "🎨 GENERAL"
            
            # Select model
            selected_model = self.models.get(model, self.models["flux"])
            
            # Generate image
            response = self.client.text_to_image(
                prompt=enhanced_prompt,
                model=selected_model
            )
            
            status = f"✅ Generated | {specialty_info} | Model: {model.upper()}"
            return response, status, detected_specialty if enhance else "general"
            
        except Exception as e:
            # Fallback to stable diffusion
            try:
                response = self.client.text_to_image(
                    prompt=prompt,
                    model=self.models["stable_diffusion"]
                )
                return response, f"✅ Generated with Stable Diffusion (fallback)", "fallback"
            except Exception as fallback_error:
                return None, f"❌ Generation failed: {str(fallback_error)}", "error"

# Initialize image generator
print("🔄 Initializing ASTROGEO Image Generator...")
image_generator = ASTROGEOImageGenerator(os.getenv('OPENAI_API_KEY'))  # Using your existing env var
print("✅ ASTROGEO Image Generator ready!")


🔄 Initializing ASTROGEO Image Generator...
✅ ASTROGEO Image Generator ready!


In [52]:
class EnhancedASTROGEOBot:
    """Enhanced ASTROGEO bot with integrated image generation"""
    
    def __init__(self, existing_smart_bot, image_gen):
        self.smart_bot = existing_smart_bot  # Your existing smart_bot function
        self.image_gen = image_gen
        
    def process_message_with_images(self, message, history):
        """Process message and determine if image generation is needed"""
        try:
            # Image generation triggers
            image_triggers = [
                "image", "picture", "photo", "visualize", "visualization", "show me", 
                "generate", "create", "draw", "display", "diagram", "chart", "map"
            ]
            
            # Check if user wants an image
            needs_image = any(trigger in message.lower() for trigger in image_triggers)
            
            # Get text response from your existing smart_bot
            text_response = self.smart_bot(message, history)
            
            if needs_image:
                # Generate image
                image, status, specialty = self.image_gen.generate_image(
                    prompt=message,
                    model="flux",
                    enhance=True
                )
                
                # Enhanced text response
                enhanced_response = f"{text_response}\n\n📸 **{status}**"
                
                return enhanced_response, image
            else:
                return text_response, None
                
        except Exception as e:
            return f"❌ Error processing message: {str(e)}", None

# Initialize enhanced bot using your existing smart_bot function
enhanced_astrogeo_bot = EnhancedASTROGEOBot(smart_bot, image_generator)
print("✅ Enhanced ASTROGEO Bot with image generation ready!")


✅ Enhanced ASTROGEO Bot with image generation ready!


In [55]:
def create_enhanced_gradio_interface():
    """Create enhanced Gradio interface with image generation support"""
    
    # Custom CSS for better appearance
    custom_css = """
    .gradio-container {
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif !important;
        max-width: 1200px !important;
        margin: 0 auto !important;
    }
    .image-container {
        border: 2px solid #e1e1e1;
        border-radius: 10px;
        padding: 10px;
        margin: 10px 0;
    }
    .specialty-astronomy {
        background: linear-gradient(45deg, #1a1a3e, #4a90e2);
        color: white;
        padding: 10px;
        border-radius: 8px;
        margin: 5px;
    }
    .specialty-geospatial {
        background: linear-gradient(45deg, #1e4d2b, #27ae60);
        color: white;
        padding: 10px;
        border-radius: 8px;
        margin: 5px;
    }
    """
    
    with gr.Blocks(css=custom_css, theme="soft", title="🛰️ ASTROGEO AI Enhanced") as interface:
        
        # Enhanced header
        gr.HTML("""
        <div style='text-align: center; padding: 25px; background: linear-gradient(45deg, #1e3c72, #2a5298, #27ae60); border-radius: 15px; margin-bottom: 20px;'>
            <h1 style='color: white; margin-bottom: 10px; font-size: 2.5em;'>🛰️ ASTROGEO AI ENHANCED</h1>
            <p style='color: #e8f4ff; font-size: 1.2em; margin-bottom: 15px;'><strong>Multi-Platform Earth Observation + Image Generation</strong></p>
            <div style='display: flex; justify-content: center; gap: 20px; flex-wrap: wrap;'>
                <div style='background: rgba(255,255,255,0.2); padding: 12px; border-radius: 8px; min-width: 150px;'>
                    <div style='font-size: 1.5em; margin-bottom: 5px;'>🌌</div>
                    <div style='font-weight: bold;'>ASTRONOMY</div>
                    <div style='font-size: 0.9em;'>Deep Space • Telescopes</div>
                </div>
                <div style='background: rgba(255,255,255,0.2); padding: 12px; border-radius: 8px; min-width: 150px;'>
                    <div style='font-size: 1.5em; margin-bottom: 5px;'>🛰️</div>
                    <div style='font-weight: bold;'>GEOSPATIAL</div>
                    <div style='font-size: 0.9em;'>Earth Observation • GIS</div>
                </div>
                <div style='background: rgba(255,255,255,0.2); padding: 12px; border-radius: 8px; min-width: 150px;'>
                    <div style='font-size: 1.5em; margin-bottom: 5px;'>🎨</div>
                    <div style='font-weight: bold;'>IMAGE AI</div>
                    <div style='font-size: 0.9em;'>Advanced Visualization</div>
                </div>
            </div>
        </div>
        """)
        
        # Main chat interface
        with gr.Row():
            with gr.Column(scale=2):
                chatbot = gr.Chatbot(
                    label="ASTROGEO AI Assistant",
                    height=500,
                    bubble_full_width=False,
                    show_label=True
                )
                
                with gr.Row():
                    msg = gr.Textbox(
                        label="Your Message",
                        placeholder="Ask about MOSDAC, VEDAS, BHUVAN data or request visualizations...",
                        lines=3,
                        scale=4,
                        show_label=False
                    )
                    submit_btn = gr.Button("🚀 Send", variant="primary", scale=1, size="lg")
            
            with gr.Column(scale=1):
                # Image output area
                image_output = gr.Image(
                    label="Generated Visualization",
                    height=400,
                    visible=False,
                    show_label=True,
                    container=True
                )
                
                # Status display
                status_display = gr.HTML(
                    value="<div style='text-align: center; padding: 20px; color: #666;'>💭 Ready for your questions...</div>",
                    visible=True
                )
        
        # Enhanced examples section
        with gr.Row():
            with gr.Column():
                gr.HTML("<h3 style='color: #4a90e2; text-align: center;'>🌌 ASTRONOMY EXAMPLES</h3>")
                gr.Examples(
                    examples=[
                        "Show me an image of the Andromeda Galaxy",
                        "What are the different types of nebulae? Show me one.",
                        "Create a visualization of Jupiter's moons",
                        "Generate an image of the James Webb Space Telescope",
                        "Visualize a black hole with accretion disk"
                    ],
                    inputs=msg,
                    label="Astronomy Examples"
                )
            
            with gr.Column():
                gr.HTML("<h3 style='color: #27ae60; text-align: center;'>🛰️ GEOSPATIAL EXAMPLES</h3>")
                gr.Examples(
                    examples=[
                        "Show me MOSDAC satellite data visualization",
                        "Create an image of forest monitoring from VEDAS",
                        "Generate BHUVAN mapping interface visualization",
                        "Show me INSAT-3D weather satellite imagery",
                        "Visualize Landsat agricultural monitoring"
                    ],
                    inputs=msg,
                    label="Geospatial Examples"
                )
        
        # Settings section
        with gr.Accordion("⚙️ Advanced Settings", open=False):
            with gr.Row():
                model_choice = gr.Dropdown(
                    choices=["flux", "stable_diffusion", "recraft"],
                    value="flux",
                    label="Image Generation Model",
                    info="FLUX: Best quality | Stable Diffusion: Most reliable | Recraft: Technical diagrams"
                )
                enhance_prompts = gr.Checkbox(
                    value=True,
                    label="Enhanced Prompts",
                    info="Automatically optimize prompts for better results"
                )
        
        # Chat functionality with image support
        def respond_with_images(message, history, model, enhance):
            try:
                # Update image generator settings
                enhanced_astrogeo_bot.image_gen.current_model = model
                
                # Process message
                text_response, generated_image = enhanced_astrogeo_bot.process_message_with_images(message, history)
                
                # Update chat history
                history.append((message, text_response))
                
                # Update status and image display
                if generated_image:
                    # Determine specialty for status display
                    specialty = enhanced_astrogeo_bot.image_gen.classify_image_type(message)
                    
                    if specialty == "astronomy":
                        status_html = """
                        <div class='specialty-astronomy' style='text-align: center;'>
                            <h3>🌌 ASTRONOMY MODE ACTIVATED</h3>
                            <p>Generated cosmic visualization with astronomical accuracy</p>
                        </div>
                        """
                    else:
                        status_html = """
                        <div class='specialty-geospatial' style='text-align: center;'>
                            <h3>🛰️ GEOSPATIAL MODE ACTIVATED</h3>
                            <p>Generated Earth observation visualization</p>
                        </div>
                        """
                    
                    return (
                        history, 
                        "", 
                        generated_image, 
                        gr.update(visible=True), 
                        status_html
                    )
                else:
                    # Text-only response
                    status_html = "<div style='text-align: center; padding: 15px; background: #f8f9fa; border-radius: 8px;'><h4>💬 Text Response Generated</h4><p>No image generation requested</p></div>"
                    
                    return (
                        history, 
                        "", 
                        None, 
                        gr.update(visible=False), 
                        status_html
                    )
                    
            except Exception as e:
                error_msg = f"❌ Error: {str(e)}"
                history.append((message, error_msg))
                error_html = f"<div style='color: red; text-align: center; padding: 10px;'><strong>Error:</strong> {str(e)}</div>"
                
                return (
                    history, 
                    "", 
                    None, 
                    gr.update(visible=False), 
                    error_html
                )
        
        # Event handlers
                # Event handlers
        submit_btn.click(
            respond_with_images,
            inputs=[msg, chatbot, model_choice, enhance_prompts],
            outputs=[chatbot, msg, image_output, image_output, status_display],
            queue=True
        )
        
        msg.submit(
            respond_with_images,
            inputs=[msg, chatbot, model_choice, enhance_prompts],
            outputs=[chatbot, msg, image_output, image_output, status_display],
            queue=True
        )
    
    return interface

# Create and store the enhanced interface
enhanced_interface = create_enhanced_gradio_interface()
print("✅ Enhanced Gradio interface created!")



C:\Users\LENOVO\AppData\Local\Temp\ipykernel_636\826338290.py:63: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_636\826338290.py:63: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


✅ Enhanced Gradio interface created!


In [56]:
from huggingface_hub import InferenceClient
from IPython.display import Image, display
import base64
from io import BytesIO
from PIL import Image as PILImage

class ASTROGEOImageGenerator:
    """Enhanced image generator with fal.ai compatible models"""
    
    def __init__(self, hf_token):
        self.client = InferenceClient(
            provider="fal-ai",
            api_key=hf_token
        )
        
        # ✅ FAL.AI COMPATIBLE MODELS ONLY
        self.models = {
            "flux": "black-forest-labs/FLUX.1-dev",     # ✅ Works with fal.ai
            "flux_schnell": "black-forest-labs/FLUX.1-schnell",  # ✅ Faster FLUX
            "recraft": "recraft-ai/recraft-v3",         # ✅ Technical diagrams
            "aura": "fal-ai/aura-flow",                 # ✅ Alternative model
        }
        
        # Keywords for smart classification
        self.astronomy_keywords = [
            "star", "galaxy", "nebula", "planet", "cosmos", "universe", "telescope", 
            "constellation", "black hole", "solar system", "asteroid", "comet",
            "space", "celestial", "astronomical", "cosmic", "lunar", "mars", "jupiter"
        ]
        
        self.geospatial_keywords = [
            "satellite", "earth", "landsat", "sentinel", "modis", "radar", "lidar",
            "remote sensing", "gis", "mapping", "terrain", "land use", "forest",
            "agriculture", "urban", "mosdac", "vedas", "bhuvan", "insat", "weather"
        ]
    
    def classify_image_type(self, prompt):
        """Classify if prompt is astronomy or geospatial focused"""
        prompt_lower = prompt.lower()
        
        astro_score = sum(1 for keyword in self.astronomy_keywords if keyword in prompt_lower)
        geo_score = sum(1 for keyword in self.geospatial_keywords if keyword in prompt_lower)
        
        if astro_score > geo_score:
            return "astronomy"
        elif geo_score > astro_score:
            return "geospatial"
        else:
            return "geospatial"  # Default to geospatial for earth observation focus
    
    def enhance_prompt(self, prompt, specialty="auto"):
        """Enhance prompt based on specialty"""
        if specialty == "auto":
            specialty = self.classify_image_type(prompt)
        
        if specialty == "astronomy":
            enhanced = f"High-resolution astronomical photograph of {prompt}, deep space imagery, cosmic details, professional space telescope quality, detailed celestial objects, stunning cosmic colors, 4K resolution"
        else:  # geospatial
            enhanced = f"High-resolution satellite imagery of {prompt}, earth observation photography, remote sensing visualization, technical accuracy, geographic detail, professional satellite photography, clear atmospheric view"
        
        return enhanced, specialty
    
    def generate_image(self, prompt, model="flux", enhance=True):
        """Generate image with fal.ai compatible models"""
        try:
            # Enhance prompt if requested
            if enhance:
                enhanced_prompt, detected_specialty = self.enhance_prompt(prompt)
                specialty_info = f"🌌 ASTRONOMY" if detected_specialty == "astronomy" else "🛰️ GEOSPATIAL"
            else:
                enhanced_prompt = prompt
                specialty_info = "🎨 GENERAL"
            
            # Select working model
            selected_model = self.models.get(model, self.models["flux"])
            
            # Generate image
            response = self.client.text_to_image(
                prompt=enhanced_prompt,
                model=selected_model
            )
            
            status = f"✅ Generated | {specialty_info} | Model: {model.upper()}"
            return response, status, detected_specialty if enhance else "general"
            
        except Exception as e:
            # Try different models in order
            fallback_models = ["flux_schnell", "aura", "recraft"]
            
            for fallback_model in fallback_models:
                if fallback_model != model:  # Don't retry the same model
                    try:
                        response = self.client.text_to_image(
                            prompt=enhanced_prompt if enhance else prompt,
                            model=self.models[fallback_model]
                        )
                        return response, f"✅ Generated with {fallback_model.upper()} (fallback)", "fallback"
                    except:
                        continue
            
            # If all models fail
            return None, f"❌ All models failed. Error: {str(e)}", "error"

# Initialize image generator with corrected models
print("🔄 Initializing ASTROGEO Image Generator with fal.ai compatible models...")
image_generator = ASTROGEOImageGenerator(os.getenv('HF_TOKEN'))
print("✅ ASTROGEO Image Generator ready with working models!")


🔄 Initializing ASTROGEO Image Generator with fal.ai compatible models...
✅ ASTROGEO Image Generator ready with working models!


In [57]:
def create_enhanced_gradio_interface():
    """Create enhanced Gradio interface with image generation support"""
    
    # Custom CSS for better appearance
    custom_css = """
    .gradio-container {
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif !important;
        max-width: 1200px !important;
        margin: 0 auto !important;
    }
    .image-container {
        border: 2px solid #e1e1e1;
        border-radius: 10px;
        padding: 10px;
        margin: 10px 0;
    }
    .specialty-astronomy {
        background: linear-gradient(45deg, #1a1a3e, #4a90e2);
        color: white;
        padding: 10px;
        border-radius: 8px;
        margin: 5px;
    }
    .specialty-geospatial {
        background: linear-gradient(45deg, #1e4d2b, #27ae60);
        color: white;
        padding: 10px;
        border-radius: 8px;
        margin: 5px;
    }
    """
    
    with gr.Blocks(css=custom_css, theme="soft", title="🛰️ ASTROGEO AI Enhanced") as interface:
        
        # Enhanced header
        gr.HTML("""
        <div style='text-align: center; padding: 25px; background: linear-gradient(45deg, #1e3c72, #2a5298, #27ae60); border-radius: 15px; margin-bottom: 20px;'>
            <h1 style='color: white; margin-bottom: 10px; font-size: 2.5em;'>🛰️ ASTROGEO AI ENHANCED</h1>
            <p style='color: #e8f4ff; font-size: 1.2em; margin-bottom: 15px;'><strong>Multi-Platform Earth Observation + Image Generation</strong></p>
            <div style='display: flex; justify-content: center; gap: 20px; flex-wrap: wrap;'>
                <div style='background: rgba(255,255,255,0.2); padding: 12px; border-radius: 8px; min-width: 150px;'>
                    <div style='font-size: 1.5em; margin-bottom: 5px;'>🌌</div>
                    <div style='font-weight: bold;'>ASTRONOMY</div>
                    <div style='font-size: 0.9em;'>Deep Space • Telescopes</div>
                </div>
                <div style='background: rgba(255,255,255,0.2); padding: 12px; border-radius: 8px; min-width: 150px;'>
                    <div style='font-size: 1.5em; margin-bottom: 5px;'>🛰️</div>
                    <div style='font-weight: bold;'>GEOSPATIAL</div>
                    <div style='font-size: 0.9em;'>Earth Observation • GIS</div>
                </div>
                <div style='background: rgba(255,255,255,0.2); padding: 12px; border-radius: 8px; min-width: 150px;'>
                    <div style='font-size: 1.5em; margin-bottom: 5px;'>🎨</div>
                    <div style='font-weight: bold;'>IMAGE AI</div>
                    <div style='font-size: 0.9em;'>Advanced Visualization</div>
                </div>
            </div>
        </div>
        """)
        
        # Main chat interface
        with gr.Row():
            with gr.Column(scale=2):
                chatbot = gr.Chatbot(
                    label="ASTROGEO AI Assistant",
                    height=500,
                    bubble_full_width=False,
                    show_label=True
                )
                
                with gr.Row():
                    msg = gr.Textbox(
                        label="Your Message",
                        placeholder="Ask about MOSDAC, VEDAS, BHUVAN data or request visualizations...",
                        lines=3,
                        scale=4,
                        show_label=False
                    )
                    submit_btn = gr.Button("🚀 Send", variant="primary", scale=1, size="lg")
            
            with gr.Column(scale=1):
                # Image output area
                image_output = gr.Image(
                    label="Generated Visualization",
                    height=400,
                    visible=False,
                    show_label=True,
                    container=True
                )
                
                # Status display
                status_display = gr.HTML(
                    value="<div style='text-align: center; padding: 20px; color: #666;'>💭 Ready for your questions...</div>",
                    visible=True
                )
        
        # Enhanced examples section
        with gr.Row():
            with gr.Column():
                gr.HTML("<h3 style='color: #4a90e2; text-align: center;'>🌌 ASTRONOMY EXAMPLES</h3>")
                gr.Examples(
                    examples=[
                        "Show me an image of the Andromeda Galaxy",
                        "What are the different types of nebulae? Show me one.",
                        "Create a visualization of Jupiter's moons",
                        "Generate an image of the James Webb Space Telescope",
                        "Visualize a black hole with accretion disk"
                    ],
                    inputs=msg,
                    label="Astronomy Examples"
                )
            
            with gr.Column():
                gr.HTML("<h3 style='color: #27ae60; text-align: center;'>🛰️ GEOSPATIAL EXAMPLES</h3>")
                gr.Examples(
                    examples=[
                        "Show me MOSDAC satellite data visualization",
                        "Create an image of forest monitoring from VEDAS",
                        "Generate BHUVAN mapping interface visualization",
                        "Show me INSAT-3D weather satellite imagery",
                        "Visualize Landsat agricultural monitoring"
                    ],
                    inputs=msg,
                    label="Geospatial Examples"
                )
        
        # Settings section with CORRECTED model choices
        with gr.Accordion("⚙️ Advanced Settings", open=False):
            with gr.Row():
                model_choice = gr.Dropdown(
                    choices=["flux", "flux_schnell", "recraft", "aura"],  # ✅ CORRECTED CHOICES
                    value="flux",
                    label="Image Generation Model",
                    info="FLUX: Best quality | FLUX Schnell: Faster | Recraft: Technical diagrams | Aura: Alternative"
                )
                enhance_prompts = gr.Checkbox(
                    value=True,
                    label="Enhanced Prompts",
                    info="Automatically optimize prompts for better results"
                )
        
        # Chat functionality with image support
        def respond_with_images(message, history, model, enhance):
            try:
                # Process message with enhanced bot
                text_response, generated_image = enhanced_astrogeo_bot.process_message_with_images(message, history)
                
                # Update chat history
                history.append((message, text_response))
                
                # Update status and image display
                if generated_image:
                    # Determine specialty for status display
                    specialty = enhanced_astrogeo_bot.image_gen.classify_image_type(message)
                    
                    if specialty == "astronomy":
                        status_html = """
                        <div class='specialty-astronomy' style='text-align: center;'>
                            <h3>🌌 ASTRONOMY MODE ACTIVATED</h3>
                            <p>Generated cosmic visualization with astronomical accuracy</p>
                        </div>
                        """
                    else:
                        status_html = """
                        <div class='specialty-geospatial' style='text-align: center;'>
                            <h3>🛰️ GEOSPATIAL MODE ACTIVATED</h3>
                            <p>Generated Earth observation visualization</p>
                        </div>
                        """
                    
                    return (
                        history, 
                        "", 
                        generated_image, 
                        gr.update(visible=True), 
                        status_html
                    )
                else:
                    # Text-only response
                    status_html = "<div style='text-align: center; padding: 15px; background: #f8f9fa; border-radius: 8px;'><h4>💬 Text Response Generated</h4><p>No image generation requested</p></div>"
                    
                    return (
                        history, 
                        "", 
                        None, 
                        gr.update(visible=False), 
                        status_html
                    )
                    
            except Exception as e:
                error_msg = f"❌ Error: {str(e)}"
                history.append((message, error_msg))
                error_html = f"<div style='color: red; text-align: center; padding: 10px;'><strong>Error:</strong> {str(e)}</div>"
                
                return (
                    history, 
                    "", 
                    None, 
                    gr.update(visible=False), 
                    error_html
                )
        
        # Event handlers
        submit_btn.click(
            respond_with_images,
            inputs=[msg, chatbot, model_choice, enhance_prompts],
            outputs=[chatbot, msg, image_output, image_output, status_display],
            queue=True
        )
        
        msg.submit(
            respond_with_images,
            inputs=[msg, chatbot, model_choice, enhance_prompts],
            outputs=[chatbot, msg, image_output, image_output, status_display],
            queue=True
        )
    
    return interface

# Create and store the enhanced interface
enhanced_interface = create_enhanced_gradio_interface()
print("✅ Enhanced Gradio interface created with fal.ai compatible models!")


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_636\3179263850.py:63: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_636\3179263850.py:63: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


✅ Enhanced Gradio interface created with fal.ai compatible models!


In [61]:
# Test the enhanced system with sample queries
def test_enhanced_system():
    """Test both text and image generation capabilities"""
    
    test_queries = [
        # Text-only queries (should not generate images)
        "What types of data are available in MOSDAC?",
        "How do I access VEDAS services?",
        
        # Image generation queries (should generate images)
        "Show me a visualization of the Andromeda Galaxy",
        "Generate an image of INSAT-3D satellite monitoring weather",
        "Create a diagram of remote sensing workflow"
    ]
    
    print("🧪 Testing Enhanced ASTROGEO AI System...")
    print("="*50)
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n🔍 Test {i}: {query}")
        print("-" * 40)
        
        try:
            # Test the enhanced bot
            text_response, image = enhanced_astrogeo_bot.process_message_with_images(query, [])
            
            print(f"📝 Text Response: {text_response[:200]}...")
            
            if image:
                print(f"🖼️ Image: Generated successfully!")
                # You can display the image here if needed
                # display(image)
            else:
                print(f"💭 Image: None (text-only response)")
                
        except Exception as e:
            print(f"❌ Error: {e}")
    
    print(f"\n✅ Testing complete!")

# Uncomment to run tests
# test_enhanced_system()

print("🎉 ASTROGEO AI Enhanced System is fully operational!")
print("📊 System Features:")
print("   ✅ Multi-platform RAG (MOSDAC + VEDAS + BHUVAN)")
print("   ✅ Advanced image generation (Astronomy + Geospatial)")  
print("   ✅ Smart specialty detection")
print("   ✅ Professional Gradio interface")
print("   ✅ Error handling and fallbacks")
print("   ✅ Conversation memory")
print("   ✅ Source attribution")


🎉 ASTROGEO AI Enhanced System is fully operational!
📊 System Features:
   ✅ Multi-platform RAG (MOSDAC + VEDAS + BHUVAN)
   ✅ Advanced image generation (Astronomy + Geospatial)
   ✅ Smart specialty detection
   ✅ Professional Gradio interface
   ✅ Error handling and fallbacks
   ✅ Conversation memory
   ✅ Source attribution


In [62]:
def create_complete_astrogeo_interface():
    """Create complete ASTROGEO AI interface with all features"""
    
    custom_css = """
    .gradio-container {
        font-family: 'Segoe UI', sans-serif !important;
        max-width: 1400px !important;
        margin: 0 auto !important;
    }
    .specialty-astronomy {
        background: linear-gradient(45deg, #1a1a3e, #4a90e2);
        color: white; padding: 15px; border-radius: 10px; text-align: center;
    }
    .specialty-geospatial {
        background: linear-gradient(45deg, #1e4d2b, #27ae60);
        color: white; padding: 15px; border-radius: 10px; text-align: center;
    }
    """
    
    with gr.Blocks(css=custom_css, theme=gr.themes.Soft(), title="🛰️ ASTROGEO AI Enhanced") as interface:
        
        # Header
        gr.HTML("""
        <div style='text-align: center; padding: 25px; background: linear-gradient(45deg, #1e3c72, #2a5298, #27ae60); border-radius: 15px; margin-bottom: 20px;'>
            <h1 style='color: white; font-size: 2.5em; margin-bottom: 10px;'>🛰️ ASTROGEO AI ENHANCED</h1>
            <p style='color: #e8f4ff; font-size: 1.2em; margin-bottom: 15px;'>
                <strong>Multi-Platform Earth Observation + AI Assistant</strong>
            </p>
            <div style='display: flex; justify-content: center; gap: 20px; flex-wrap: wrap;'>
                <div style='background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px; min-width: 180px;'>
                    <div style='font-size: 1.8em;'>🌌</div>
                    <div style='font-weight: bold;'>ASTRONOMY</div>
                    <div style='font-size: 0.9em;'>Deep Space Objects</div>
                </div>
                <div style='background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px; min-width: 180px;'>
                    <div style='font-size: 1.8em;'>🛰️</div>
                    <div style='font-weight: bold;'>GEOSPATIAL</div>
                    <div style='font-size: 0.9em;'>Earth Observation</div>
                </div>
                <div style='background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px; min-width: 180px;'>
                    <div style='font-size: 1.8em;'>🤖</div>
                    <div style='font-weight: bold;'>AI ASSISTANT</div>
                    <div style='font-size: 0.9em;'>Smart RAG System</div>
                </div>
            </div>
        </div>
        """)
        
        # Main Layout
        with gr.Row():
            # Chat Column
            with gr.Column(scale=3):
                chatbot = gr.Chatbot(
                    label="🤖 ASTROGEO AI Assistant",
                    height=550,
                    bubble_full_width=False,
                    show_label=True,
                    avatar_images=("👤", "🛰️")
                )
                
                with gr.Row():
                    msg = gr.Textbox(
                        placeholder="Ask about MOSDAC, VEDAS, BHUVAN data or space topics...",
                        lines=3,
                        scale=4,
                        show_label=False
                    )
                    with gr.Column(scale=1):
                        send_btn = gr.Button("🚀 Send", variant="primary", size="lg")
                        clear_btn = gr.Button("🗑️ Clear", variant="secondary")
            
            # Status Column
            with gr.Column(scale=2):
                status_display = gr.HTML(
                    value="""
                    <div style='text-align: center; padding: 20px; background: #f8f9fa; border-radius: 10px; border: 1px solid #dee2e6;'>
                        <h3 style='color: #495057; margin-bottom: 10px;'>💭 Ready for Questions</h3>
                        <p style='color: #6c757d; margin: 0;'>Ask about space, satellites, or earth observation!</p>
                    </div>
                    """,
                    visible=True
                )
        
        # Examples Section
        with gr.Row():
            with gr.Column():
                gr.HTML("<h3 style='color: #4a90e2; text-align: center;'>🌌 ASTRONOMY EXAMPLES</h3>")
                gr.Examples(
                    examples=[
                        "What are pulsars and neutron stars?",
                        "Tell me about the James Webb Space Telescope",
                        "What are the different types of galaxies?",
                        "Explain black holes and event horizons",
                        "What is the lifecycle of a star?"
                    ],
                    inputs=msg
                )
            
            with gr.Column():
                gr.HTML("<h3 style='color: #27ae60; text-align: center;'>🛰️ GEOSPATIAL EXAMPLES</h3>")
                gr.Examples(
                    examples=[
                        "What types of data are available in MOSDAC?",
                        "How do I access VEDAS services?",
                        "What is BHUVAN and its applications?",
                        "Tell me about INSAT-3D satellite data",
                        "How to download satellite imagery?"
                    ],
                    inputs=msg
                )
        
        # Chat Functions
        def respond(message, history):
            if message.strip():
                try:
                    # Use your existing smart_bot function
                    response = smart_bot(message, history)
                    
                    # Determine response type for status
                    if any(word in message.lower() for word in ["star", "galaxy", "planet", "space", "cosmic"]):
                        status_html = """
                        <div class='specialty-astronomy'>
                            <h3>🌌 ASTRONOMY MODE</h3>
                            <p>Providing astronomical knowledge and insights</p>
                        </div>
                        """
                    elif any(word in message.lower() for word in ["satellite", "mosdac", "vedas", "bhuvan", "earth"]):
                        status_html = """
                        <div class='specialty-geospatial'>
                            <h3>🛰️ GEOSPATIAL MODE</h3>
                            <p>Accessing earth observation knowledge base</p>
                        </div>
                        """
                    else:
                        status_html = """
                        <div style='text-align: center; padding: 15px; background: #f8f9fa; border-radius: 10px;'>
                            <h4>💬 General Response</h4>
                            <p>Providing comprehensive information</p>
                        </div>
                        """
                    
                    history.append((message, response))
                    return history, "", status_html
                    
                except Exception as e:
                    error_response = f"❌ Error: {str(e)}"
                    history.append((message, error_response))
                    error_status = f"<div style='color: red; padding: 10px; text-align: center;'><strong>Error occurred</strong></div>"
                    return history, "", error_status
            
            return history, message, status_display.value
        
        def clear_chat():
            return [], ""
        
        # Event Handlers
        send_btn.click(respond, [msg, chatbot], [chatbot, msg, status_display])
        msg.submit(respond, [msg, chatbot], [chatbot, msg, status_display])
        clear_btn.click(clear_chat, outputs=[chatbot, msg])
    
    return interface

# Create interface
enhanced_interface = create_complete_astrogeo_interface()
print("✅ Enhanced interface created!")


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_636\1749286029.py:53: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_636\1749286029.py:53: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


✅ Enhanced interface created!


In [65]:
# Launch the enhanced interface
print("🚀 Launching Enhanced ASTROGEO AI Interface...")

try:
    enhanced_interface.launch(
        share=True,              # Creates public link
        server_port=7863,        # Different port from your existing one
        server_name="0.0.0.0",   # Network accessible
        show_error=True,
        inbrowser=True,          # Auto-open in browser
        auth=None                # No authentication
    )
    
    print("✅ Enhanced ASTROGEO AI Interface launched successfully!")
    print("🌐 Access your interface at: http://127.0.0.1:7863")
    print("🔗 Public link will be generated automatically")
    
except Exception as e:
    print(f"❌ Launch error: {e}")
    print("🔄 Trying alternative port...")
    
    # Fallback launch
    enhanced_interface.launch(
        share=True,
        server_port=7864,
        show_error=True
    )


🚀 Launching Enhanced ASTROGEO AI Interface...
Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
* Running on public URL: https://2c5a189c3c7854fd95.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


✅ Enhanced ASTROGEO AI Interface launched successfully!
🌐 Access your interface at: http://127.0.0.1:7863
🔗 Public link will be generated automatically


In [64]:
# Add HF_TOKEN and Image Generator Integration
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-hugging-face-token')

# Simple Image Generator
from huggingface_hub import InferenceClient

def generate_image_if_requested(message, response):
    """Check if image is requested and generate it"""
    image_triggers = ["generate", "image", "show me", "visualize", "create", "picture", "photo"]
    
    if any(trigger in message.lower() for trigger in image_triggers):
        try:
            # Initialize client
            client = InferenceClient(provider="fal-ai", api_key=os.getenv('HF_TOKEN'))
            
            # Generate image
            image = client.text_to_image(
                prompt=f"High-resolution astronomical image of {message}",
                model="black-forest-labs/FLUX.1-dev"
            )
            
            enhanced_response = f"{response}\n\n📸 **✅ Generated astronomical visualization!**"
            return enhanced_response, image
            
        except Exception as e:
            enhanced_response = f"{response}\n\n📸 **❌ Image generation failed: {str(e)}**"
            return enhanced_response, None
    
    return response, None

# Update your interface response function
def updated_respond(message, history):
    if message.strip():
        try:
            # Get text response
            text_response = smart_bot(message, history)
            
            # Try to generate image if requested
            enhanced_response, generated_image = generate_image_if_requested(message, text_response)
            
            history.append((message, enhanced_response))
            
            # Return status based on generation
            if generated_image:
                status_html = """
                <div class='specialty-astronomy'>
                    <h3>🌌 IMAGE GENERATED!</h3>
                    <p>Created astronomical visualization</p>
                </div>
                """
            else:
                status_html = """
                <div style='text-align: center; padding: 15px; background: #f8f9fa; border-radius: 10px;'>
                    <h4>💬 Text Response</h4>
                    <p>Comprehensive information provided</p>
                </div>
                """
            
            return history, "", status_html
            
        except Exception as e:
            error_response = f"❌ Error: {str(e)}"
            history.append((message, error_response))
            return history, "", "<div style='color: red;'>Error occurred</div>"
    
    return history, message, ""

print("✅ Image generation integration ready!")


✅ Image generation integration ready!
